
<h2> Movie Recommendation System</h2>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import load
from numpy import savez_compressed
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
import os
import ast
from ast import literal_eval
from datetime import datetime
import seaborn as sns
from scipy import stats
from surprise.model_selection import cross_validate, KFold
import warnings; warnings.simplefilter('ignore')

In [2]:
movies=pd.read_csv('movies_metadata.csv',nrows=9099)
ratings=pd.read_csv('ratings_small.csv')
links=pd.read_csv('links.csv')



In [3]:
movies.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194


In [4]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int64  
 1   imdbId   45843 non-null  int64  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


In [5]:
links=links.rename({"movieId":"id"},axis='columns')

<h2>Simple Recommender</h2>


The implementation of this model is extremely trivial. All we have to do is sort our movies based on ratings and popularity and display the top movies of our list. As an added step, we can pass in a genre argument to get the top movies of a particular genre.

In [6]:
movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [7]:
ratings=ratings.rename({"movieId":"id"},axis='columns')

In [8]:
df1 = pd.DataFrame(ratings)
df2=pd.DataFrame(movies['title'])
merge=[df1,df2]
result=pd.concat(merge)

In [9]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109103 entries, 0 to 9098
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  float64
 1   id         100004 non-null  float64
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  float64
 4   title      9099 non-null    object 
dtypes: float64(4), object(1)
memory usage: 5.0+ MB


In [10]:
vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
k = vote_averages.mean()
k

5.687877788767996

I use the TMDB Ratings to come up with our **Top Movies Chart.** I will use IMDB's *weighted rating* formula to construct my chart. Mathematically, it is represented as follows:

Weighted Rating (WR) = $(\frac{t}{t + m} . s) + (\frac{l}{t + l} . k)$

where,
* *t* is the number of votes for the movie
* *l* is the minimum votes required to be listed in the chart
* *s* is the average rating of the movie
* *k* is the mean vote across the whole report

The next step is to determine an appropriate value for *m*, the minimum votes required to be listed in the chart. We will use **95th percentile** as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the movies in the list.

I will build our overall Top 250 Chart and will define a function to build charts for a particular genre.

In [11]:
l= vote_counts.quantile(0.95)
l


867.2000000000007

In [12]:
shortlisted = movies[(movies['vote_count'] >= l) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())][['title',  'vote_count', 'vote_average', 'genres']]
shortlisted ['vote_count'] = shortlisted['vote_count'].astype('int')
shortlisted ['vote_average'] = shortlisted['vote_average'].astype('int')
shortlisted .shape

(455, 4)

In [13]:
def weighted_rating(z):
    t = z['vote_count']#v
    s= z['vote_average']#r
    return (t/(t+l) * s) + (l/(l+t) * k)

In [14]:
shortlisted['wr'] = shortlisted.apply(weighted_rating, axis=1)

In [15]:
shortlisted= shortlisted.sort_values('wr', ascending=False).head(250)

<h5>Top Movies</h5>

In [16]:
shortlisted.head(15)

,title,vote_count,vote_average,genres,wr
2843,Fight Club,9678,8,[Drama],7.809859
4863,The Lord of the Rings: The Fellowship of the Ring,8892,8,"[Adventure, Fantasy, Action]",7.794545
292,Pulp Fiction,8670,8,"[Thriller, Crime]",7.789763
314,The Shawshank Redemption,8358,8,"[Drama, Crime]",7.782653
7000,The Lord of the Rings: The Return of the King,8226,8,"[Adventure, Fantasy, Action]",7.779498
351,Forrest Gump,8147,8,"[Comedy, Drama, Romance]",7.777565
5814,The Lord of the Rings: The Two Towers,7641,8,"[Adventure, Fantasy, Action]",7.764336
256,Star Wars,6778,8,"[Adventure, Action, Science Fiction]",7.737734
1225,Back to the Future,6239,8,"[Adventure, Comedy, Science Fiction, Family]",7.717842
834,The Godfather,6024,8,"[Drama, Crime]",7.709039


In [17]:
b = movies.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
b.name = 'genre'
gen_movies= movies.drop('genres', axis=1).join(b)

In [18]:
def build_chart(genre, percentile=0.85):
    df = gen_movies[gen_movies['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    k = vote_averages.mean()
    l = vote_counts.quantile(percentile)
    
    shortlisted = df[(df['vote_count'] >= l) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title',  'vote_count', 'vote_average']]
    shortlisted['vote_count'] = shortlisted['vote_count'].astype('int')
    shortlisted['vote_average'] = shortlisted['vote_average'].astype('int')
    
    shortlisted['wr'] = shortlisted.apply(lambda x: (x['vote_count']/(x['vote_count']+l) * x['vote_average']) + (l/(l+x['vote_count']) * k), axis=1)
    shortlisted = shortlisted.sort_values('wr', ascending=False).head(250)
    
    return shortlisted

In [56]:
build_chart('Horror')

,title,vote_count,vote_average,wr
1213,The Shining,3890,8,7.829775
1176,Psycho,2405,8,7.735290
1171,Alien,4564,7,6.909440
1158,Aliens,3282,7,6.876758
1338,Jaws,2628,7,6.848950
...,...,...,...,...
1612,Alien: Resurrection,1388,5,5.062106
5189,Jason X,298,4,4.656925
4605,Ghosts of Mars,299,4,4.655771
8240,Seed of Chucky,302,4,4.652334


In [20]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9099 entries, 0 to 9098
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9099 non-null   bool   
 1   belongs_to_collection  1300 non-null   object 
 2   budget                 9099 non-null   int64  
 3   genres                 9099 non-null   object 
 4   homepage               573 non-null    object 
 5   id                     9099 non-null   int64  
 6   imdb_id                9098 non-null   object 
 7   original_language      9099 non-null   object 
 8   original_title         9099 non-null   object 
 9   overview               9073 non-null   object 
 10  popularity             9099 non-null   float64
 11  poster_path            9068 non-null   object 
 12  production_companies   9099 non-null   object 
 13  production_countries   9099 non-null   object 
 14  release_date           9094 non-null   object 
 15  reve

<h2>Collaborative Filtering</h2>

Collaborative filtering uses similarities between users and items simultaneously to provide recommendations. This allows for serendipitous recommendations; that is, collaborative filtering models can recommend an item to user A based on the interests of a similar user B.

In [21]:
ratings.head()

,userId,id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [22]:
ratings_with_title=ratings.merge(movies,on='id')
ratings_with_title

,userId,id,rating,timestamp,adult,belongs_to_collection,budget,genres,homepage,imdb_id,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,1,1371,2.5,1260759135,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,[Drama],NaN,tt0084602,...,1982-05-28,270000000,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894
1,4,1371,4.0,949810302,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,[Drama],NaN,tt0084602,...,1982-05-28,270000000,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894
2,7,1371,3.0,851869160,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,[Drama],NaN,tt0084602,...,1982-05-28,270000000,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894
3,19,1371,4.0,855193404,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,[Drama],NaN,tt0084602,...,1982-05-28,270000000,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894
4,21,1371,3.0,853852263,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,[Drama],NaN,tt0084602,...,1982-05-28,270000000,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28743,652,26843,5.0,1440269953,False,NaN,35000000,"[Action, History, War]",NaN,tt0081059,...,1981-04-17,0,173.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,He was a man of honor in a war without any.,Lion of the Desert,False,7.5,36
28744,653,2103,3.0,948161066,False,NaN,47000000,"[Drama, Science Fiction, Mystery, Romance]",NaN,tt0307479,...,2002-11-27,30002758,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,How far will you go for a second chance?,Solaris,False,5.9,378
28745,659,167,4.0,836137550,False,NaN,48000000,"[Drama, Science Fiction]",http://www.k-pax.com/,tt0272152,...,2001-10-22,50315140,120.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Change the way you look at the world.,K-PAX,False,7.1,710
28746,659,563,3.0,834694187,False,"{'id': 10522, 'name': 'Starship Troopers Colle...",105000000,"[Adventure, Action, Thriller, Science Fiction]",NaN,tt0120201,...,1997-11-06,121214377,129.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The only good bug is a dead bug.,Starship Troopers,False,6.7,1584


In [23]:
num_rating_df=ratings_with_title.groupby('title').count()['rating'].reset_index()
num_rating_df.rename(columns={'rating':'num_rating'},inplace=True)
num_rating_df

,title,num_rating
0,...And God Created Woman,1
1,10 Things I Hate About You,7
2,11'09''01 - September 11,1
3,12 Angry Men,1
4,15 Minutes,5
...,...,...
1392,Zombie Flesh Eaters,1
1393,Zombie Holocaust,3
1394,eXistenZ,12
1395,xXx,28


In [24]:
average_rating_df=ratings_with_title.groupby('title').mean()['rating'].reset_index()
average_rating_df.rename(columns={'rating':'average_rating'},inplace=True)
average_rating_df

,title,average_rating
0,...And God Created Woman,4.000000
1,10 Things I Hate About You,2.642857
2,11'09''01 - September 11,2.000000
3,12 Angry Men,4.000000
4,15 Minutes,4.000000
...,...,...
1392,Zombie Flesh Eaters,4.000000
1393,Zombie Holocaust,3.166667
1394,eXistenZ,4.208333
1395,xXx,3.517857


In [25]:
popular_df=num_rating_df.merge(average_rating_df,on='title')
popular_df

,title,num_rating,average_rating
0,...And God Created Woman,1,4.000000
1,10 Things I Hate About You,7,2.642857
2,11'09''01 - September 11,1,2.000000
3,12 Angry Men,1,4.000000
4,15 Minutes,5,4.000000
...,...,...,...
1392,Zombie Flesh Eaters,1,4.000000
1393,Zombie Holocaust,3,3.166667
1394,eXistenZ,12,4.208333
1395,xXx,28,3.517857


In [26]:
popular_df=popular_df[popular_df['num_rating']>=100].sort_values('average_rating',ascending=False).head(50)

In [27]:
popular_df=popular_df.merge(movies,on='title').drop_duplicates('title')[['id','title','genres','num_rating','average_rating']]

In [28]:
popular_df

,id,title,genres,num_rating,average_rating
0,858,Sleepless in Seattle,"[Comedy, Drama, Romance]",200,4.487500
1,318,The Million Dollar Hotel,"[Drama, Thriller]",311,4.487138
2,527,Once Were Warriors,[Drama],244,4.303279
3,913,The Thomas Crown Affair,"[Drama, Crime, Romance]",179,4.287709
5,608,Men in Black II,"[Action, Adventure, Comedy, Science Fiction]",224,4.256696
6,296,Terminator 3: Rise of the Machines,"[Action, Thriller, Science Fiction]",324,4.256173
7,111,Scarface,"[Action, Crime, Drama, Thriller]",118,4.224576
9,260,The 39 Steps,"[Action, Thriller, Mystery]",291,4.221649
10,750,Murder She Said,"[Drama, Crime, Mystery, Comedy]",105,4.209524
11,1213,The Talented Mr. Ripley,"[Thriller, Crime, Drama]",131,4.202290


In [29]:
x=ratings_with_title.groupby("userId").count()['rating']>50
rated_users=x[x].index

In [30]:
filtered_ratings=ratings_with_title[ratings_with_title['userId'].isin(rated_users)]

In [31]:
y=filtered_ratings.groupby('title').count()['rating']>=3
famous_movies=y[y].index

In [32]:
famous_movies

Index(['10 Things I Hate About You', '15 Minutes',
       '20,000 Leagues Under the Sea', '2001: A Space Odyssey', '2010',
       '21 Grams', '24 Hour Party People', '25th Hour', '28 Days Later',
       '40 Days and 40 Nights',
       ...
       'Young Adam', 'Young Black Stallion', 'Young Frankenstein',
       'Young and Innocent', 'Z', 'Zatoichi', 'Zazie dans le métro',
       'Zombie Holocaust', 'eXistenZ', 'xXx'],
      dtype='object', name='title', length=897)

In [33]:
final_ratings=filtered_ratings[filtered_ratings['title'].isin(famous_movies)]

In [34]:
pt=final_ratings.pivot_table(index='title',columns='userId',values='rating')

In [35]:
pt.fillna(0,inplace=True)

In [36]:
pt

userId,4,15,17,19,21,22,23,30,34,36,...,641,646,647,648,649,652,654,659,664,665
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
15 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"20,000 Leagues Under the Sea",3.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,4.5,3.0,0.0,3.0
2010,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zatoichi,0.0,5.0,0.0,4.0,0.0,0.0,4.5,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zazie dans le métro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zombie Holocaust,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
similarity_score=cosine_similarity(pt)

In [39]:
similarity_score.shape

(897, 897)

In [48]:
def recommend(movie_name):
    #index fetch 
    index=np.where(pt.index==movie_name)[0][0]
    similar_movies=sorted(list(enumerate (similarity_score[index])),key=lambda x:x[1],reverse=True)[1:6]
    for i in similar_movies:
        print(pt.index[i[0]])
    

In [58]:
recommend('15 Minutes')

The Piano Teacher
The Whole Nine Yards
Adaptation.
Ice Age
A Star Is Born


In [45]:
build_chart('Action')

,title,vote_count,vote_average,wr
4863,The Lord of the Rings: The Fellowship of the Ring,8892,8,7.876390
7000,The Lord of the Rings: The Return of the King,8226,8,7.866929
5814,The Lord of the Rings: The Two Towers,7641,8,7.857338
256,Star Wars,6778,8,7.840364
1154,The Empire Strikes Back,5998,8,7.821128
...,...,...,...,...
6388,Terminator 3: Rise of the Machines,2177,5,5.100997
4239,Lara Croft: Tomb Raider,2235,5,5.098835
6417,Spy Kids 3-D: Game Over,525,4,4.742932
7939,Catwoman,833,4,4.568025
